## 데이터셋 준비 하기
Instruction Fine Tuning을 위해 데이터셋을 준비합니다.  
데이터셋은 아래의 순서로 진행하려 합니다.
---
Instruction Tuning의 데이터셋은 어떻게 만들어야할까요?  
학습에 필요한 Instruction Dataset은 아래와 같은 데이터 형태입니다.
([Stanford Alpca 데이터셋](https://github.com/tatsu-lab/stanford_alpaca/blob/main/alpaca_data.json)을 참조했습니다)
```plain
질문 :
건강하기 위한 3가지 팁 알려줘.
답변 :
1. 충분한 양의 채소와 과일을 섭취해서 균형있는 식단을 가지세요.
2. 신체가 활동적이고 건강할 수 있도록 규칙적으로 운동을 하세요.
3. 충분한 수면시간을 가지고 규칙적인 수면 습관을 가지세요.
```
데이터셋을 만드는건 많은 시간과 비용이 소요되기 때문에, 강력한 언어모델을 활용하게 됩니다. 이 예제에서는 ChatGPT를 활용하여 학습 데이터를 생성하였습니다.  
데이터셋은 1) 키워드 기반으로 생성, 2) 부동산 스터디 카페글을 기반으로 생성하는 2가지 방법으로 생성하였습니다.  

이 예제에서 데이터셋을 만드는 순서는 간단하게 아래와 같습니다.  

#### <span style="color: #F2D388;"> 1. 도메인 정하기  </span>
어떤 Assistant(챗봇)을 만들까?를 먼저 정해봅니다. 사내 봇? 주식 답변? ...  
이 예제에서는 부동산을 타겟으로 하였습니다.  


#### <span style="color: #F2D388;"> 2. 키워드 기반으로 데이터셋 만들기 </span>
##### <span style="color: #ECA75D;"> 2-1. 부동산에 관련된 키워드 모으기 </span>
책, 뉴스, 부동산 사이트에서 키워드를 수집해보았습니다.  
사람들이 궁금해할만한 키워드를 대상으로 합니다. (예: 전세 계약, 신혼부부 특별공급, 토지거래허가구역...)  
키워드에 대한 예제 데이터는 [data/seed_words.txt](https://github.com/aiqwe/instruction-tuning-with-rag-example/blob/main/data/seed_words.txt)를 참조해주세요.  
##### <span style="color: #ECA75D;"> 2-2. 부동산 키워드를 통해 사람들이 찾아볼만한 질문리스트를 만들기 </span>
2번에서 수집한 키워드를 기반으로, 사람들이 궁금해할만한 질문리스트를 ChatGPT를 활용해 만들어봅니다.  
검색 질문리스트 예제 데이터는 [data/questions_search.jsonl](https://github.com/aiqwe/instruction-tuning-with-rag-example/blob/main/data/questions_search.jsonl)를 참조해주세요. 


#### <span style="color: #F2D388;"> 3. 카페글 기반으로 데이터셋 만들기 </span>
##### <span style="color: #ECA75D;"> 3-1. 네이버 카페 글의 질문으로 질문 데이터셋 만들기 </span>
부동산 관련 카페중 가장 큰 규모인 부동산스터디 카페의 회원간 묻고 답하기 글을 크롤링합니다.  
##### <span style="color: #ECA75D;"> 3-2. 수집한 카페 글을 유사도 기준으로 필터링하기 </span>
카페글 중엔 부동산과 관련이 없는 데이터셋이 있을 수 있습니다. 따라서 그러한 데이터를 필터링합니다.  
수집된 데이터를 그대로 ChatGPT로 필터링하면 요금이 비싸기 때문에 먼저 인코더 모델로 유사도를 계산합니다.
##### <span style="color: #ECA75D;"> 3-3. 유사도로 필터링한 데이터를 ChatGPT로 한번 더 필터링하기 </span>
3-2번에서 필터링한 데이터를 ChatGPT로 한번 더 필터링합니다. 이 때 유사도 기반으로 필터링하지 못한 데이터들을 ChatGPT가 필터링해줄 수 있습니다.  

    
#### <span style="color: #F2D388;"> 4. 질문리스트로 네이버에 검색하여 인기 글 데이터 수집하기 </span>
2, 3번에서 수집한 질문리스트를 Selenium 라이브러리를 활용하여 네이버에 검색합니다. 검색 결과중 인기글의 텍스트 데이터를 추출합니다.  
인기글 문서 예제 데이터는 [data/documents.jsonl](https://github.com/aiqwe/instruction-tuning-with-rag-example/blob/main/data/documents.jsonl)를 참조해주세요.  


#### <span style="color: #F2D388;"> 5. encoder 모델을 활용하여 데이터를 유사도 기준으로 정렬하기 </span>
4번에서 수집한 인기글 텍스트 데이터가 질문리스트와 얼마나 유사한지 계산합니다.
질문리스트와 가장 유사한 인기글 텍스트를 상위 순위로 정렬합니다.
인기글 문서 예제 데이터는 [data/documents.jsonl](https://github.com/aiqwe/instruction-tuning-with-rag-example/blob/main/data/documents.jsonl)를 참조해주세요.  


#### <span style="color: #F2D388;"> 6. Instuction 데이터셋 만들기 </span>
질문리스트 + 정렬한 인기글을 합쳐서 ChatGPT에 Instruction 데이터를 만들어달라고 요청합니다. 이 Instruction 데이터는 Fine Tuning에 사용됩니다.
인기글 문서 예제 데이터는 [data/instructions.jsonl](https://github.com/aiqwe/instruction-tuning-with-rag-example/blob/main/data/instructions.jsonl)를 참조해주세요.  

In [1]:
!pip install --quiet\
selenium==4.20.0\
openai==1.23.6\
python-dotenv==1.0.1


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


## <span style="color: #F2D388;"> 2. 키워드 기반으로 데이터셋 만들기 </span>

### <span style="color: #ECA75D;"> 2-1. 부동산에 관련된 키워드 모으기 </span>
부동산에 관련된 키워드를 수집하여 저장합니다.  
예제 데이터는 [data/seed_words.txt](https://github.com/aiqwe/instruction-tuning-with-rag-example/blob/main/data/seed_words.txt)를 참조해주세요.

In [2]:
with open("./data/seed_words.txt", "r") as f:
    seed_words = f.readlines()

In [3]:
seed_words[:4]

['전세 계약\n', '임대차 계약\n', '전세 사기\n', '임대차 분쟁\n']

불필요한 newline이 있어서 제거해줍니다.

In [4]:
# Element마다 있는 \n 제거, '전세 계약\n' -> '전세 계약'
seed_words = list(map(lambda x: x.strip("\n"), seed_words))

In [5]:
seed_words[:4]

['전세 계약', '임대차 계약', '전세 사기', '임대차 분쟁']

### <span style="color: #ECA75D;"> 2-2. 부동산 키워드를 통해 사람들이 찾아볼만한 질문리스트를 만들기 </span>
미리 작성된 프롬프트에 `seed_words`를 `format` 적용하여 프롬프트를 완성시킵니다.  
프롬프트 작성은 [Deeplearning.ai의 강의](https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers)(무료)를 참조하였습니다.

In [6]:
from prompts import SEED_WORD_PROMPT_PREFIX, SEED_WORD_PROMPT_CONTENT

# prompts.py에 프롬프트들 템플릿이 있습니다.
print(SEED_WORD_PROMPT_PREFIX + SEED_WORD_PROMPT_CONTENT)

당신은 부동산에 관심이 많은 사람입니다. 당신의 역할에 따라 주어진 seed_word에 대해 궁금해할 질문을 생성하세요.
seed_word는 총 5개씩 주어집니다.
seed_word에 대해 각각 20개의 질문을 생성해야합니다.
생성할 질문에 대한 요구 사항은 다음과 같습니다:
1. 지시대명사를 사용해서는 안됩니다. seed_word에 있는 명사를 그대로 사용하세요.
2. 이미 만들어낸 질문과 동일하거나 유사한 질문을 만들어내서는 안됩니다.
3. 만들어내는 질문들은 어휘의 다양성을 위해 다양한 단어를 사용해야 합니다.
4. 만들어내는 질문들은 문장의 다양성을 위해 의문문과 평서문을 모두 사용해야 합니다.
5. 반드시 한글로 질문을 만드세요.
6. 만들어낸 질문은 JSON형식을 따라야 하고, indent는 없어야 합니다.
7. 응답하는 답변 문자에는 줄바꿈, \n, \t, \b 등의 특수 문자가 없어야합니다.
8. seed_word에 대해 중복으로 질문을 생성했는지 확인하세요. seed_word에 대한 질문을 이미 생성했다면, 동일한 seed_word에 대한 작업을 해서는 안됩니다.
9. 아래 양식으로 출력하세요:
{{"seed_word": "주어진 seed_word", "answer": ["1번째 질문", "2번째 질문"... , "20번째 질문"]}}

seed_word는 다음과 같습니다:
{seed_word}


In [7]:
print(SEED_WORD_PROMPT_PREFIX + SEED_WORD_PROMPT_CONTENT.format(seed_word=seed_words[0]))

당신은 부동산에 관심이 많은 사람입니다. 당신의 역할에 따라 주어진 seed_word에 대해 궁금해할 질문을 생성하세요.
seed_word는 총 5개씩 주어집니다.
seed_word에 대해 각각 20개의 질문을 생성해야합니다.
생성할 질문에 대한 요구 사항은 다음과 같습니다:
1. 지시대명사를 사용해서는 안됩니다. seed_word에 있는 명사를 그대로 사용하세요.
2. 이미 만들어낸 질문과 동일하거나 유사한 질문을 만들어내서는 안됩니다.
3. 만들어내는 질문들은 어휘의 다양성을 위해 다양한 단어를 사용해야 합니다.
4. 만들어내는 질문들은 문장의 다양성을 위해 의문문과 평서문을 모두 사용해야 합니다.
5. 반드시 한글로 질문을 만드세요.
6. 만들어낸 질문은 JSON형식을 따라야 하고, indent는 없어야 합니다.
7. 응답하는 답변 문자에는 줄바꿈, \n, \t, \b 등의 특수 문자가 없어야합니다.
8. seed_word에 대해 중복으로 질문을 생성했는지 확인하세요. seed_word에 대한 질문을 이미 생성했다면, 동일한 seed_word에 대한 작업을 해서는 안됩니다.
9. 아래 양식으로 출력하세요:
{{"seed_word": "주어진 seed_word", "answer": ["1번째 질문", "2번째 질문"... , "20번째 질문"]}}

seed_word는 다음과 같습니다:
전세 계약


GPT모델의 토큰 수 계산은 Input도 포함됩니다. 따라서 불필요하게 `SEED_WORD_PROMPT_PREFIX`를 모든 키워드에 반복해서 전달할 필요가 없습니다.  

In [8]:
# 모델에게 Prefix 토큰을 중복해서 전달할 필요가 없으므로, Prefix 토큰은 1번만 사용하고 Content을 반복해서 프롬프트를 생성
seed_word_prefix = SEED_WORD_PROMPT_PREFIX
seed_word_content = [SEED_WORD_PROMPT_CONTENT.format(seed_word=s) for s in seed_words]

In [9]:
# 1개의 Prefix 마다 n_content개의 Content를 추가합니다.
n_content = 5

seed_word_prompts = []
total_prompts = len(seed_words) // n_content
for idx in range(1, total_prompts+2):
    start_index = (idx -1) * n_content
    end_index = idx * n_content
    seed_word_prompt = seed_word_prefix + "\n".join(seed_word_content[start_index:end_index])
    seed_word_prompts.append(seed_word_prompt)

In [10]:
print(seed_word_prompts[0])

당신은 부동산에 관심이 많은 사람입니다. 당신의 역할에 따라 주어진 seed_word에 대해 궁금해할 질문을 생성하세요.
seed_word는 총 5개씩 주어집니다.
seed_word에 대해 각각 20개의 질문을 생성해야합니다.
생성할 질문에 대한 요구 사항은 다음과 같습니다:
1. 지시대명사를 사용해서는 안됩니다. seed_word에 있는 명사를 그대로 사용하세요.
2. 이미 만들어낸 질문과 동일하거나 유사한 질문을 만들어내서는 안됩니다.
3. 만들어내는 질문들은 어휘의 다양성을 위해 다양한 단어를 사용해야 합니다.
4. 만들어내는 질문들은 문장의 다양성을 위해 의문문과 평서문을 모두 사용해야 합니다.
5. 반드시 한글로 질문을 만드세요.
6. 만들어낸 질문은 JSON형식을 따라야 하고, indent는 없어야 합니다.
7. 응답하는 답변 문자에는 줄바꿈, \n, \t, \b 등의 특수 문자가 없어야합니다.
8. seed_word에 대해 중복으로 질문을 생성했는지 확인하세요. seed_word에 대한 질문을 이미 생성했다면, 동일한 seed_word에 대한 작업을 해서는 안됩니다.
9. 아래 양식으로 출력하세요:
{{"seed_word": "주어진 seed_word", "answer": ["1번째 질문", "2번째 질문"... , "20번째 질문"]}}

seed_word는 다음과 같습니다:
전세 계약
임대차 계약
전세 사기
임대차 분쟁
깡통 전세


이제 ChatGPT에게 데이터를 작성하도록 합니다.  
💡 ChatGPT API를 사용하기 전에, [ChatGPT 모델 가격 정책](https://openai.com/pricing)을 참조하세요.  
ChatGPT는 지속적으로 업데이트를 하는데, 과거버전은 사용하지 않도록 합니다.  
과거버젼은 퍼포먼스도 안좋은데 가격이 비싸므로 사용할 이유가 없습니다.  
이 예제에서는 `gpt-4-turbo`를 사용합니다.

데이터 전처리의 경우 병렬처리를 사용합니다.  
spark, hive 등의 분산 처리 시스템 또는 Multi Threading과 같은 병렬 처리를 적극 활용하면 속도가 많이 개선됩니다.  
이 예제의 ChatGPT API는 [Multi Threading](https://docs.python.org/ko/3/library/concurrent.futures.html#concurrent.futures.ThreadPoolExecutor)을 사용합니다.

In [13]:
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

# Worker의 수는 32개 또는 코어수 +4개가 디폴트값입니다. - 공식 가이드 문서 참조
with ThreadPoolExecutor() as pool:
    questions = list(tqdm(pool.map(partial(get_completion, model="gpt-4-turbo"), seed_word_prompts)))
    
with open("./data/questions_keyword.jsonl", "a") as f:
    f.write("\n".join(questions))

55it [14:13, 15.53s/it]


In [17]:
from utils import jload

jload("./data/questions_keyword.jsonl")[0]

{'seed_word': '전세 계약',
 'answer': ['전세 계약을 체결할 때 필요한 서류는 무엇인가요?',
  '전세 계약 시 주의해야 할 점은 무엇인가요?',
  '전세 계약 갱신 청구권은 어떻게 적용되나요?',
  '전세 계약을 해지하려면 어떤 절차를 밟아야 하나요?',
  '전세 계약 시 보증금 반환 문제는 어떻게 해결하나요?',
  '전세 계약 기간 중 집주인이 바뀌면 어떻게 되나요?',
  '전세 계약을 할 때 중개수수료는 얼마인가요?',
  '전세 계약 시 보증금을 보호받을 수 있는 방법은 무엇인가요?',
  '전세 계약 갱신 시 임대료 인상률은 어떻게 결정되나요?',
  '전세 계약 체결 전에 확인해야 할 주택 상태는 무엇인가요?',
  '전세 계약 시 임차인의 권리와 의무는 무엇인가요?',
  '전세 계약 시 임대인의 권리와 의무는 무엇인가요?',
  '전세 계약 시 확정일자는 왜 중요한가요?',
  '전세 계약 시 전입신고는 언제 해야 하나요?',
  '전세 계약 갱신 거절 시 임차인은 어떤 조치를 취할 수 있나요?',
  '전세 계약 시 보증금을 전액 돌려받지 못하는 경우는 어떤 경우인가요?',
  '전세 계약 시 임대인이 파산하면 보증금은 어떻게 되나요?',
  '전세 계약 시 임대인과 임차인 간의 분쟁 해결 방법은 무엇인가요?',
  '전세 계약 시 임대인이 계약을 일방적으로 해지할 수 있나요?',
  '전세 계약 시 임차인이 보증금을 늦게 받는 경우 어떻게 대처해야 하나요?']}

## <span style="color: #F2D388;"> 3. 카페글 기반으로 데이터셋 만들기 </span>

### <span style="color: #ECA75D;"> 3-1. 네이버 카페 글의 질문으로 질문 데이터셋 만들기 </span>
부동산 관련 카페중 가장 큰 규모인 부동산스터디 카페의 회원간 묻고 답하기 글을 크롤링합니다.  

In [2]:
from utils import get_document_through_selenium

cafe_data = get_document_through_selenium(
    crawling_type="cafe",
    n_page=200,
    indent=4,
    mode="a",
    save_path = "./data/questions_cafe.json"
)

In [16]:
jload("./data/questions_cafe.json")[0]['document'][:10]

['[LG전자 베스트샵 봉천센트럴점]🥳RENEWAL OPEN 행사🥳 (광고)',
 '[LG전자 베스트샵 롯데 수원점] 리뉴얼 GRAND OPEN 가전제품 세일 💕',
 '부동산 스터디 카페에 올라오는 주식 홍보방들은 모두 사기글입니다.',
 '[공지]붇스터디게시판 운영가이드(2023.9.3 수정추가)',
 '부동산 스터디 카페 내에 계정 해킹 사례가 너무 많네요',
 '전라도 사람들은 투표권 주지말자',
 '고수님들은 3억 자금이 있다면 무엇을 하시나요?',
 '용산 vs 강남 신혼부부 집 조언부탁드립니다!',
 '골프연습장(닭장) 비거리가 실제보다 짧나요',
 '중앙대 vs 이화여대']

### <span style="color: #ECA75D;"> 3-2. 수집한 카페 글을 유사도 기준으로 필터링하기 </span>
카페글 중엔 부동산과 관련이 없는 데이터셋이 있을 수 있습니다. 따라서 그러한 데이터를 필터링해야합니다.  
단순히 "부동산" 이라는 단어와 유사도를 계산하는것 보다, `seed_keywords.txt`의 모든 키워드와 유사도가 있는지를 확인해 보는게 더 다양한 데이터를 수집할 수 있을 것 같습니다.  
따라서, `seed_keywords.txt`와의 유사도를 계산한 후, 특정 점수 이상의 데이터만 수집하도록 하겠습니다.  

In [19]:
from utils import jload

questions_cafe = jload("./data/questions_cafe.json")

In [20]:
questions_cafe[0]['document'][:20]

['[LG전자 베스트샵 봉천센트럴점]🥳RENEWAL OPEN 행사🥳 (광고)',
 '[LG전자 베스트샵 롯데 수원점] 리뉴얼 GRAND OPEN 가전제품 세일 💕',
 '부동산 스터디 카페에 올라오는 주식 홍보방들은 모두 사기글입니다.',
 '[공지]붇스터디게시판 운영가이드(2023.9.3 수정추가)',
 '부동산 스터디 카페 내에 계정 해킹 사례가 너무 많네요',
 '전라도 사람들은 투표권 주지말자',
 '고수님들은 3억 자금이 있다면 무엇을 하시나요?',
 '용산 vs 강남 신혼부부 집 조언부탁드립니다!',
 '골프연습장(닭장) 비거리가 실제보다 짧나요',
 '중앙대 vs 이화여대',
 '공수처 일하나 보네요',
 '생애최초 대출 활용하여 주택 구입 문의, 후보지 추천 포함',
 '[공유] [단독] 부산 7위 신태양건설도 부도… 줄도산 현실화',
 '석열이형~~',
 '같은 동네 매도매수 거래 문의합니다.',
 '현금3억 투자, 가양6단지 vs 자양2동 한강변모아타운 추진지',
 '신생아 신혼부부 평촌 초원마을한양 vs 범계 은하수 벽산,신성',
 '성동구 12-13억대 투자할만한 아파트',
 '시세 20억집에 대출이 5억이면 반전세 보증금 얼마가 적절한가요?',
 '부부간 증여후 자녀상속 질문']

`List[Dict[List]]`의 중첩된 형태이므로, flatten하여 list 형태로 바꿔줍니다.

In [21]:
from itertools import chain

documents = [q['document'] for q in questions_cafe]
questions_cafe = list(chain(*documents))

약 1만개의 게시글 타이틀이 수집되어 있습니다.

In [22]:
len(questions_cafe)

10008

`sort_by_similarity` 함수는 query와 documents를 받아서 유사도를 계산해주는 함수입니다.  
아래의 예시를 보면 이해가 쉽습니다.

In [5]:
from similarity import sort_by_similarity

query = "부동산"
documents = [
    "부동산 매수하려는데 조언좀...",
    "노원구 아파트 어떤가요?"
]
documents, scores = sort_by_similarity(query=query, documents=documents)

print(f"documents: \n{documents}")
print(f"scores: \n{scores}")

config.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

documents: 
['부동산 매수하려는데 조언좀...', '노원구 아파트 어떤가요?']
scores: 
[0.8845638036727905, 0.8322739601135254]


In [5]:
with open("./data/seed_words.txt", "r") as f:
    seed_words = f.readlines()
seed_words = list(map(lambda x: x.strip("\n"), seed_words))

10000개의 수집한 데이터 마다 270개의 seed_words와의 유사도를 구하려고 합니다.  
그리고 유사도 점수다 높은 데이터만 필터링하려 합니다.  
하지만 이 예제는 CPU또는 낮은 리소스의 GPU를 사용한다고 가정하기 때문에, 10000개의 데이터를 한번에 입력하면 OOM이 발생하기 쉽습니다.  
따라서 1000개씩 데이터를 나누고 유사도를 구해보겠습니다.  

In [23]:
# questions_cafe 데이터를 1000개 단위로 분리합니다
n_questions = 1000

questions_cafe_split = []
batch = len(questions_cafe) // n_questions
for idx in range(1, batch+2):
    start_index = (idx -1) * n_questions
    end_index = idx * n_questions
    questions_cafe_split.append(questions_cafe[start_index:end_index])

In [24]:
print(f"num_batch: {len(questions_cafe_split)}")
print(f"batch_size: {len(questions_cafe_split[0])}")

num_batch: 11
batch_size: 1000


In [10]:
from similarity import sort_by_similarity
from tqdm import tqdm

filtered_questions_cafe = []
threshold = 0.8
for q in questions_cafe_split:
    for w in tqdm(seed_words):
        documents, scores = sort_by_similarity(query=word, documents=q)
        documents = [d for d, s in zip(search_docs, scores) if s >= threshold]
        filtered_questions_cafe.append(documents)

### <span style="color: #ECA75D;"> 3-3. 유사도로 필터링한 데이터를 ChatGPT로 한번 더 필터링하기 </span>
수집한 데이터에서 정치적 발언, 혐오 발언 등의 데이터가 있을 수 있습니다.  
ChatGPT에 요청하여 이러한 데이터들을 필터링할 수 있게 합니다.

In [25]:
from prompts import CAFE_FILTER_PROMPT

In [26]:
print(CAFE_FILTER_PROMPT)

아래 documents들중 다음 규칙을 적용하여 답변하세요.
seed_words와 documents는 리스트 형태로 주어집니다.
요구사항은 다음과 같습니다:
1. 정치적 발언, 혐오 발언 등 독성있고 해로운 documents는 삭제하세요.
2. 리스트 안에 문자열 형태로 주어진 seed_words들과 관련이 있는 데이터만 출력하세요.
3. 출력한 데이터는 JSON형식을 따라야하고, indent는 없어야 합니다.
4. 아래 양식으로 출력하세요.
["document1", "document2", "document3" ...]  

###seed_words:
{seed_words}

###documents:
{documents}




이번에는 1개의 프롬프트에 100개의 documents를 추가해줍니다.

In [69]:
n_queries = 50

queries = []
batch = len(filtered_questions_cafe) // n_queries
for idx in range(1, batch+2):
    start_index = (idx -1) * n_queries
    end_index = idx * n_queries
    query_documents = filtered_questions_cafe[start_index:end_index]
    query_documents = "\n".join(query_documents)
    queries.append(CAFE_FILTER_PROMPT.format(seed_words=seed_words, documents=query_documents))

In [102]:
print(queries[0][:350] + "\n...\n\n" + queries[0][2432:2702] + "\n...")

아래 documents들중 다음 규칙을 적용하여 답변하세요.
seed_words와 documents는 리스트 형태로 주어집니다.
요구사항은 다음과 같습니다:
1. 정치적 발언, 혐오 발언 등 독성있고 해로운 documents는 삭제하세요.
2. 리스트 안에 문자열 형태로 주어진 seed_words들과 관련이 있는 데이터만 출력하세요.
3. 출력한 데이터는 JSON형식을 따라야하고, indent는 없어야 합니다.
4. 아래 양식으로 출력하세요.
["document1", "document2", "document3" ...]  

###seed_words:
전세 계약
임대차 계약
전세 사기
임대차 분쟁
깡통 전세
전세자금대출
...

###documents:
[LG전자 베스트샵 봉천센트럴점]🥳RENEWAL OPEN 행사🥳 (광고)
[LG전자 베스트샵 롯데 수원점] 리뉴얼 GRAND OPEN 가전제품 세일 💕
부동산 스터디 카페에 올라오는 주식 홍보방들은 모두 사기글입니다.
[공지]붇스터디게시판 운영가이드(2023.9.3 수정추가)
부동산 스터디 카페 내에 계정 해킹 사례가 너무 많네요
전라도 사람들은 투표권 주지말자
고수님들은 3억 자금이 있다면 무엇을 하시나요?
용산 vs 강남 신혼부부 집 조언부탁드립니다!
...


In [ ]:
from concurrent.futures import ThreadPoolExecutor
from functools import partial
import os

with ThreadPoolExecutor(os.cpu_count()) as pool:
    result = list(tqdm(pool.map(partial(get_completion, model="gpt-4-turbo"), queries)))
    with open("questions_cafe_filtered.jsonl", "a", encoding="utf-8") as f:
        for r in result:
            for line in r.split("\n"):
                f.write(line)
                f.write("\n")

## <span style="color: #F2D388;"> 4. 질문리스트로 네이버에 검색하여 인기 글 데이터 수집하기 </span>
3번에서 생성한 질문리스트를 selenium 라이브러리를 통해 네이버로 검색합니다.  
검색 결과의 인기글의 텍스트 정보를 저장합니다.  

In [5]:
query_data_search = jload("./data/questions_keyword.jsonl")

In [6]:
query_data_search[0]

{'seed_word': '전세 계약',
 'answer': ['전세 계약을 체결할 때 필요한 서류는 무엇인가요?',
  '전세 계약 시 주의해야 할 점은 무엇인가요?',
  '전세 계약 갱신 청구권은 어떻게 적용되나요?',
  '전세 계약을 해지하려면 어떤 절차를 밟아야 하나요?',
  '전세 계약 시 보증금 반환 문제는 어떻게 해결하나요?',
  '전세 계약 기간 중 집주인이 바뀌면 어떻게 되나요?',
  '전세 계약을 할 때 중개수수료는 얼마인가요?',
  '전세 계약 시 보증금을 보호받을 수 있는 방법은 무엇인가요?',
  '전세 계약 갱신 시 임대료 인상률은 어떻게 결정되나요?',
  '전세 계약 체결 전에 확인해야 할 주택 상태는 무엇인가요?',
  '전세 계약 시 임차인의 권리와 의무는 무엇인가요?',
  '전세 계약 시 임대인의 권리와 의무는 무엇인가요?',
  '전세 계약 시 확정일자는 왜 중요한가요?',
  '전세 계약 시 전입신고는 언제 해야 하나요?',
  '전세 계약 갱신 거절 시 임차인은 어떤 조치를 취할 수 있나요?',
  '전세 계약 시 보증금을 전액 돌려받지 못하는 경우는 어떤 경우인가요?',
  '전세 계약 시 임대인이 파산하면 보증금은 어떻게 되나요?',
  '전세 계약 시 임대인과 임차인 간의 분쟁 해결 방법은 무엇인가요?',
  '전세 계약 시 임대인이 계약을 일방적으로 해지할 수 있나요?',
  '전세 계약 시 임차인이 보증금을 늦게 받는 경우 어떻게 대처해야 하나요?']}

In [7]:
from itertools import chain

# answer를 추출해서 [[query1], [query2]] 구조를 [query1, query2]로 flatten해주기
queries = [data['answer'] for data in query_data]
queries = list(chain(*queries))

selenium이 4.1 버전으로 업그레이드 되면서 별도로 webdriver를 다운로드 받을 필요가 없어졌습니다.  
webdriver 파일이 없으면 selenium이 자체적으로 다운로드하게 된다고 합니다. [stackoverflow](https://stackoverflow.com/questions/22130109/cant-use-chrome-driver-for-seleniumhttps://stackoverflow.com/questions/22130109/cant-use-chrome-driver-for-selenium)  
본 예제의 selenium 코드 크롤링 코드는 [wikidocs](https://wikidocs.net/137914) 내용을 참조하였습니다.  
아래 코드는 내부에 `multiprocessing`으로 구현되어 있으며, pool의 갯수는 CPU Core 갯수로 설정되어 있습니다.  
코드 실행시에 PC가 조금 느려질 수 있으니 참조해주세요.

In [1]:
from utils import get_document_through_selenium

search_data = get_document_through_selenium(
    crawling_type="search"
    inputs=queries,
    n_documents=5,
    indent=4,
    mode="a",
    save_path = "./data/document.json"
)

In [3]:
jload("./data/document.json")[0]

{'question': '전세 계약을 체결할 때 필요한 서류는 무엇인가요?',
 'document': ['경매, 공매 절차에서 무엇보다 중요한 것은 바로 "배당 순위"이기 때문입니다. 집주인의 입장에서 세입자와 전세계약을 체결하고 전세보증금을 받는 것은 사실상 무이자 대출을 받는... 신탁등기 전세사기까지는 아니라고 하더라도 부동산 임대차 계약을 체결할 때 위탁자와 공인중개사가 "신탁등기를 말소해줄테니 하자"고 제안하는 경우가 많습니다. 이...',
  '금리 및 서류 1. 2024신혼부부 전세대출 대상 주택도시기금의 신혼부부 전세대출을 받으려면 위의 6가지 조건을 만족시켜야 합니다. 우선 전세계약을 체결한 이후 임차보증금의 5... 마지막으로 필요한 서류입니다. 먼저 본인 확인을 위한 신분증과 신혼부부 확인을 위한 주민등록초본이 하구요. 결혼 예정자인 경우 예식장 서 또는 청첩장이 있어야 합니다....',
  '오늘은 청년들에게 내 집 마련할 때 도움이 되고자 청년전용 버팀목 전세자금 대출에 대해서 알아보겠습니다. 청년 전용 버팀목 전세자금 대출은 당연히 청년들에게 해당되며 이... 같은 계약서 형태의 서류가 필요로 합니다. 소득확인 서류로는 소득확인 증명서, 원천징수영수증, 급여 명세표 등이 하고 사업자는 소득확인 증명서나 원천징수영수증...',
  '만약 전세보증금에서 임차인 부담금을 LH 통해서 받으려면, 추가 서류를 제출해야 한다. 어떤 서류가 필요한지는 팩스를 보내면 LH 측에서 알려주시는데, 나는 또 서류 준비하기 귀찮아서 그냥 임대인한테 바로 받겠다고 작성해서 보냈다. 이주 재계약 체결일에 이 도 들고 가야 하는데, 에 차질이 없도록 꼬옥 미리 받아두어야 한다. (사실 기존 임대인에게...',
  '부동산 전자계약 체결 시 연 0.1%p가 적용되며, 자녀 수에 따라서도 금리가 적용됩니다. 다자녀일 경우 연 0.7%p, 2자녀일 경우 연 0.5%p, 1자녀일 경우 연 0.3%p까지 적용됩니다. 부동산... 참고로 전세 계약을 새로 하거나, 갱신할 

가끔 크롤링이 안되는 경우가 있어서 한번 검수해줍니다.

In [4]:
search_data = jload('./data/document.json')
for idx in range(len(search_data)):
    if len(search_data[idx]['document']) == 0:
        print(f"Index [{idx}] 번째 데이터가 수집되지 않았네요.")

Index [1686] 번째 데이터가 수집되지 않았네요.
Index [3950] 번째 데이터가 수집되지 않았네요.
Index [3990] 번째 데이터가 수집되지 않았네요.


수집되지 않은 데이터를 한번더 크롤링하게 하고 `document.json`에 저장합니다.

In [7]:
for idx in range(len(search_data)):
    if len(search_data[idx]['document']) == 0:
        search_data[idx] = get_document_through_selenium(
            inputs=search_data[idx]['question'],
            n_documents=5,
            indent=4
        )[0]

1it [00:03,  3.71s/it]
1it [00:02,  2.77s/it]
1it [00:02,  2.78s/it]


제대로 수집되었는지 또 검사해줍니다.

In [8]:
for idx in range(len(search_data)):
    if len(search_data[idx]['document']) == 0:
        print(f"Index [{idx}] 번째 데이터가 수집되지 않았네요.")

Index [3950] 번째 데이터가 수집되지 않았네요.
Index [3990] 번째 데이터가 수집되지 않았네요.


원인 파악을 위해 해당 질문으로 직접 검색을 해봅니다.  
검색해보면 인기글이 검색되지 않습니다.  
해당 데이터는 삭제하겠습니다.  

In [11]:
search_data[3950]

{'question': '도봉구 부동산 관련 법적 요건은 무엇인가요?', 'document': []}

In [17]:
# 삭제할 인덱스 모으기
remove_idx_list = []
for idx in range(len(search_data)):
    if len(search_data[idx]['document']) == 0:
        remove_idx_list.append(idx)

# 삭제할 인덱스를 pop 
for idx in remove_idx_list:
    search_data.pop(idx)

In [18]:
from utils import jsave

jsave(data=search_data, save_path="./data/document.json", mode="w", indent=4)

In [19]:
for idx in range(len(search_data)):
    if len(search_data[idx]['document']) == 0:
        print(f"Index [{idx}] 번째 데이터가 수집되지 않았네요.")

## <span style="color: #F2D388;"> 5. encoder 모델을 활용하여 데이터를 유사도 기준으로 정렬하기 </span>
비교적 최근에 공개된 인코더 모델인 [intfloat/multilingual-e5-base](https://huggingface.co/intfloat/multilingual-e5-base) 모델([arxiv](https://arxiv.org/pdf/2402.05672))을 사용하여 유사도를 계산합니다.  
질문을 했을 때, 검색되는 인기글 데이터들중 유사도가 높은 순서대로 문서를 다시 정렬합니다.  
정렬된 순서대로 데이터를 좀더 많이 참조하도록 프롬프트를 통해 지시합니다.

In [21]:
search_data = jload("./data/document.json")

e5모델의 자세한 내용은 [hugginface](https://huggingface.co/intfloat/e5-base-v2)를 참조하세요  
유사도 계산시 `multiprocessing` 모듈을 사용합니다.  
데이터 전처리시에 `multiprocessing`, `threading`, `concurrent.futures` 등의 병렬처리 도구를 사용하는 것이 속도 개선에 많이 도움이 되었습니다.  

In [24]:
from similarity import sort_by_similarity
from tqdm import tqdm

for question in tqdm(search_data):
    question['document'], question['scores'] = sort_by_similarity(question['question'], question['document'])

100%|█████████████████████████████████████████████████████████████████████████████████████| 5417/5417 [1:46:19<00:00,  1.18s/it]


In [27]:
jsave(search_data, "./data/document.json", "w", indent=4)

## <span style="color: #F2D388;"> 6. Instuction 데이터셋 만들기 </span>
완성된 질문리스트와 인기글 데이터를 통해 ChatGPT에 정답을 출력하도록 요청합니다.  
ChatGPT는 아래의 프롬프트처럼 question에 대한 answer 답변을 출력합니다.  
완성된 question, answer 텍스트는 학습시킬 모델의 훈련용 데이터셋으로 전달됩니다.  
본 문서의 프롬프트는 [Stanford Alpaca의 프롬프트](https://github.com/tatsu-lab/stanford_alpaca/blob/main/prompt.txthttps://github.com/tatsu-lab/stanford_alpaca/blob/main/prompt.txt)를 참조하였습니다.  

In [6]:
from prompts import INSTRUCTION_PROMPT_PREFIX, INSTRUCTION_PROMPT_CONTENT

print(INSTRUCTION_PROMPT_PREFIX + INSTRUCTION_PROMPT_CONTENT)

요청받은 question을 document를 참조하여 answer로 답변하세요.
question 1개당 여러개의 document가 주어지며, question은 10개씩 전달됩니다.

요구사항은 다음과 같습니다:
1. 어휘의 다양성을 위해 같은 단어를 반복하지 않습니다.
2. 문장의 형태가 다양해야합니다. 예를 들어 질문과 명령형이 결합되는 형태여야 합니다.
3. 답변은 제공받는 document들을 기반으로 작성되어야 합니다.
4. 제공되는 document의 순서가 먼저 제공될 수록 더 중요한 데이터이므로 답변에 더 많은 영향을 끼쳐야합니다.
5. 답변은 자세한 내용이 포함되도록 제공되어야하지만 200단어를 넘지 않는 것이 좋습니다.

출력 형식은 JSON형식을 따라야 합니다.
Indentation은 없도록 출력하세요.
각 question마다 출력 형식은 다음과 같아야합니다:
{'question': '전달 받은 question의 내용', 'answer': '답변 내용'}

###question:
{question}
###document:
{document}


In [7]:
search = jload("./data/document.json")

In [8]:
# 리스트 형태인 document 데이터를 하나로 합치기
for data in search:
    data['document'] = "\n".join([f"{idx+1}. {d}" for idx, d in enumerate(data['document'])])

In [9]:
print(search[0]['document'])

1. 말해야 되나요? 주택임대차보호법 제 6조 제 1항 (계약의 갱신) 계약이 해지되기 6개월 전부터 2개월 전까지, 계약을 해지하겠다는 통보를 해야 한다. 즉, 법률상 늦어도 계약이 종료되기 2개월 전까지는 임대인에게 말해야 된다는 뜻인데요. 만약 해당 기간 안에 말하지 않았다면? 자동으로 연장되는 묵시적 갱신이 될 수 있습니다. 때문에, 종료 시점에 맞춰서 자금을...
2. 기간을 보통 2달 정도 주게 되는데, 해당 기간 내 임대인은 건설사에 중도금 대출과 잔금을 모두 상환해야 하기 때문에 금액이 떨어지게 됩니다. 오늘은, 미등기 신축 아파트 전세 계약... 요즘은, 임차인이 전세를 구할 때 전세자금 대출을 받는 경우가 상당히 많습니다. 그런데 대출에 필요한 모든 서류를 은행에 제출했을 경우 은행에서 대출금이 나오는지?는...
3. 전세보증금반환소송 유리한 결과를 위해 전세 보증금은 결국 임대인과 임차인이 계약을 하게 된 시점부터 법적 효력이 가능한 기간 동안 임차인이 임대인의 부동산을 사용하겠다는... 사례에는 얼마나 더 다양한 시점이 있는지를 사전에 꼭 파악한 뒤 철저히 준비를 해야 한다고 하였습니다. 인가결정이나도 안도할 수 없어 반환금 지급 명령은 인가 결정이 되었다고...
4. 전세사기변호사 법무법인 이현 부동산 소송 전략센터입니다. 대여금반환청구소송에 급여채권가압류, 추심까지 책임져드렸어요 생각보다 많은 분들이 겪고 있는 문제입니다. 얼마나... 수원전세사기변호사 ‘내용증명’을 통해 보다 확실하게 계약만료를 알릴 수 있습니다. 내용증명에는 집주인에게 ‘부동산 목적물’ ‘임차’ ‘만료일’ ‘종료...
5. 생각하면 전세사기 초기 때의 내 예전 모습이 떠올라서, 마음이 찡해진다... 얼마나 힘드실까... 그래서 결심했다. 그 사람들을 위해서, 정보의 사막 속에서 몸소 필요한 정보를 찾고... 어, 근데 계약기간이 남았다? 1. 이사 가고 싶다. 2. 전세금 반환 소송을 걸고 싶다. (승소 후 재산 압류, 경매 진행, 채권추심 하고 싶다.) 1, 2 

In [10]:
print(INSTRUCTION_PROMPT_CONTENT)


###question:
{question}
###document:
{document}


In [11]:
prefix = INSTRUCTION_PROMPT_PREFIX
instructions = []
total_instructions = len(search) // 10
for index in range(1, total_instructions + 2):
    start_index = (index - 1) * 10
    end_index = index * 10
    content = "\n".join([
        INSTRUCTION_PROMPT_CONTENT.format(question = data['question'], document = data['document'])
        for data in search[start_index:end_index]
    ])
    instruction = prefix + content
    instructions.append(instruction)

In [12]:
print(instructions[0])

요청받은 question을 document를 참조하여 answer로 답변하세요.
question 1개당 여러개의 document가 주어지며, question은 10개씩 전달됩니다.

요구사항은 다음과 같습니다:
1. 어휘의 다양성을 위해 같은 단어를 반복하지 않습니다.
2. 문장의 형태가 다양해야합니다. 예를 들어 질문과 명령형이 결합되는 형태여야 합니다.
3. 답변은 제공받는 document들을 기반으로 작성되어야 합니다.
4. 제공되는 document의 순서가 먼저 제공될 수록 더 중요한 데이터이므로 답변에 더 많은 영향을 끼쳐야합니다.
5. 답변은 자세한 내용이 포함되도록 제공되어야하지만 200단어를 넘지 않는 것이 좋습니다.

출력 형식은 JSON형식을 따라야 합니다.
Indentation은 없도록 출력하세요.
각 question마다 출력 형식은 다음과 같아야합니다:
{'question': '전달 받은 question의 내용', 'answer': '답변 내용'}

###question:
전세 계약 기간은 보통 얼마나 되나요?
###document:
1. 말해야 되나요? 주택임대차보호법 제 6조 제 1항 (계약의 갱신) 계약이 해지되기 6개월 전부터 2개월 전까지, 계약을 해지하겠다는 통보를 해야 한다. 즉, 법률상 늦어도 계약이 종료되기 2개월 전까지는 임대인에게 말해야 된다는 뜻인데요. 만약 해당 기간 안에 말하지 않았다면? 자동으로 연장되는 묵시적 갱신이 될 수 있습니다. 때문에, 종료 시점에 맞춰서 자금을...
2. 기간을 보통 2달 정도 주게 되는데, 해당 기간 내 임대인은 건설사에 중도금 대출과 잔금을 모두 상환해야 하기 때문에 금액이 떨어지게 됩니다. 오늘은, 미등기 신축 아파트 전세 계약... 요즘은, 임차인이 전세를 구할 때 전세자금 대출을 받는 경우가 상당히 많습니다. 그런데 대출에 필요한 모든 서류를 은행에 제출했을 경우 은행에서 대출금이 나오는지?는...
3. 전세보증금반환소송 유리한 결과를 위해 전세 보증금은 결국 임대인과 임차인이

포맷팅하여 완성한 프롬프트는 아래와 같습니다.  
아래 데이터를 ChatGPT API로 넘겨줍니다.

In [13]:
print(instructions[0][:2000])

요청받은 question을 document를 참조하여 answer로 답변하세요.
question 1개당 여러개의 document가 주어지며, question은 10개씩 전달됩니다.

요구사항은 다음과 같습니다:
1. 어휘의 다양성을 위해 같은 단어를 반복하지 않습니다.
2. 문장의 형태가 다양해야합니다. 예를 들어 질문과 명령형이 결합되는 형태여야 합니다.
3. 답변은 제공받는 document들을 기반으로 작성되어야 합니다.
4. 제공되는 document의 순서가 먼저 제공될 수록 더 중요한 데이터이므로 답변에 더 많은 영향을 끼쳐야합니다.
5. 답변은 자세한 내용이 포함되도록 제공되어야하지만 200단어를 넘지 않는 것이 좋습니다.

출력 형식은 JSON형식을 따라야 합니다.
Indentation은 없도록 출력하세요.
각 question마다 출력 형식은 다음과 같아야합니다:
{'question': '전달 받은 question의 내용', 'answer': '답변 내용'}

###question:
전세 계약 기간은 보통 얼마나 되나요?
###document:
1. 말해야 되나요? 주택임대차보호법 제 6조 제 1항 (계약의 갱신) 계약이 해지되기 6개월 전부터 2개월 전까지, 계약을 해지하겠다는 통보를 해야 한다. 즉, 법률상 늦어도 계약이 종료되기 2개월 전까지는 임대인에게 말해야 된다는 뜻인데요. 만약 해당 기간 안에 말하지 않았다면? 자동으로 연장되는 묵시적 갱신이 될 수 있습니다. 때문에, 종료 시점에 맞춰서 자금을...
2. 기간을 보통 2달 정도 주게 되는데, 해당 기간 내 임대인은 건설사에 중도금 대출과 잔금을 모두 상환해야 하기 때문에 금액이 떨어지게 됩니다. 오늘은, 미등기 신축 아파트 전세 계약... 요즘은, 임차인이 전세를 구할 때 전세자금 대출을 받는 경우가 상당히 많습니다. 그런데 대출에 필요한 모든 서류를 은행에 제출했을 경우 은행에서 대출금이 나오는지?는...
3. 전세보증금반환소송 유리한 결과를 위해 전세 보증금은 결국 임대인과 임차인이

In [14]:
from concurrent.futures import ThreadPoolExecutor
from functools import partial
import os

with ThreadPoolExecutor(os.cpu_count()) as pool:
    result = list(tqdm(pool.map(partial(utils.get_completion, model="gpt-4-turbo"), instructions[10:])))
    with open("instruction.jsonl", "a", encoding="utf-8") as f:
        for r in result:
            for line in r.split("\n"):
                f.write(line)
                f.write("\n")